In [112]:
import mido
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [184]:
#function definitions
from mido import Message, MidiFile, MidiTrack
import matplotlib.pyplot as plt
import os, copy, pickle
import numpy as np
import time


def saveData(data, dataDir):
	with open(dataDir, 'wb') as f:
		pickle.dump(data, f)

def loadData(dataDir):
	with open(dataDir, 'rb') as f:
		data = pickle.load(f)
		return data


def maxTimesteps(limitSongs):
	maxSteps = 0
	for fileName in sorted(os.listdir(dataset_path)):
		if ".mid" in fileName:
			mid = MidiFile(dataset_path + fileName) 
			numSteps = 0
			for message in mid.tracks[0]:
				numSteps += message.time
			maxSteps = max([maxSteps, numSteps])
			limitSongs -= 1
			if limitSongs == 0:
				break
	return maxSteps

def getTimesteps(dataDir, limitSongs):
	timesteps = []
	for fileName in sorted(os.listdir(dataDir)):
		if ".mid" in fileName:
			mid = MidiFile(dataDir + fileName) 
			numSteps = 0
			for message in mid.tracks[0]:
				numSteps += message.time
			timesteps.append(numSteps)
			limitSongs -= 1
			if limitSongs == 0:
				break
	return timesteps


#This function loads the .mid files and converts them to a reduced (in the time axis) piano roll representation
def createRepresentation(dataDir, limitSongs=0, reductionRatio=128):
	#perhaps it would make more sense to create a midi2roll function aside and simplify this one
	#To Do: if limitSongs is bigger than the actual maximum or is 0 we should look for the number of files in the path to determine the first dimension
	#To Do: extract notes that are triggered so that we can reduce the third dimension from 128 to a smaller value

	#timesteps = maxTimesteps(limitSongs)
	#songs = np.zeros((limitSongs, timesteps, 128))
	timesteps = getTimesteps(dataDir, limitSongs)
	songs = []
	idx = 0
	for fileName in sorted(os.listdir(dataDir)): #iterate per file
		if ".mid" in fileName: #check
			print("Loading file %d: %s" % (idx+1, fileName))
			mid = MidiFile(dataDir + fileName)
			song = np.zeros(np.array((int(np.ceil(1+timesteps[idx]/float(reductionRatio))), 128+1))) #1 additional note to denote end of track
			for i, track in enumerate(mid.tracks):
				if i != 0: #track 0 contains meta info we don't need
					ts = 0 #init time
					realts = 0
					notesOn = []
					for message in track:
						ticks = message.time #indicates delta change where next event is happening
						while ticks > 0: #advance timestep pointer to delta while we keep enabling the activated notes
							for note in notesOn:
								#songs[idx][ts][note-1] = 1
								song[ts][note] = 1
							ticks -= 1
							realts += 1
							if np.floor(realts/float(reductionRatio)) > ts:
								ts += 1

						#update state at current timestep according to message
						if message.type == 'note_on':
							notesOn.append(message.note)
						if message.type == 'note_off':
							notesOn.remove(message.note) #To do: check if ValueError is triggered

			#denote end of track
			song[-1][-1] = 1
			#add to songs
			songs.append(song)

			#check limit of songs for collection
			limitSongs -= 1
			if limitSongs == 0:
				break
			idx += 1 #next song...
			#could merge idx with limitsongs

	return songs

def thresholdOutput(x, threshold=0.5, normalize=False):
	return [0 if note < threshold else 1 for note in x]

def sampleOutput(x, normalize=False):
	if normalize:
		return [0 if np.random.uniform() > normalizeProbability(note) else 1 for note in x]
	else:
		return [0 if np.random.uniform() > note else 1 for note in x]

def normalizeProbability(prob):
	return 1/(1+np.power(10000, (-prob + 0.5)))

def roll2midi(roll, notesMap, reductionRatio=128): #roll is a (1, ts, input_dim) tensor
	mid = MidiFile()

	track = MidiTrack()
	mid.tracks.append(track)

	tones = np.zeros(roll.shape[1])
	ticks = 0
	for ts in roll:
		for i in range(ts.shape[0]-1):

			if ts[i] == 1 and tones[i] == 0:
				#record note_on event
				track.append(Message('note_on', velocity=127, note=notesMap[i], time=ticks*reductionRatio))
				tones[i] = 1
				ticks = 0

			if ts[i] == 0 and tones[i] == 1:
				#record note_off event
				track.append(Message('note_off', velocity=127, note=notesMap[i], time=ticks*reductionRatio))
				tones[i] = 0
				ticks = 0

		ticks += 1

	#last pass for notes off (end of track)	
	for i in range(roll.shape[1]):
		if tones[i] == 1:
			track.append(Message('note_off', note=notesMap[i], time=ticks*reductionRatio))
			ticks = 0

	return mid

def saveMidi(mid, path):
	mid.save("%ssong.mid" % (path))

#This function removes unnecessary notes and returns mapping of indexes to notes
def compressInputs(X, Y, notesMap=None):
	notesDel = set(range(129))
	for i in range(X.shape[0]):
		print(i)
		for j in range(X.shape[1]):
			for k in range(X.shape[2]):
				if X[i][j][k] == 1 and k in notesDel:
					notesDel.remove(k)

	#Just in case Y is not contained within X (depends on previous processing of roll to create inputs)
	for i in range(Y.shape[0]):
		for k in range(Y.shape[1]):
			if Y[i][k] == 1 and k in notesDel:
				notesDel.remove(k)

	if notesMap is not None:
		notesDel = notesDel.difference(set(notesMap))
		
	X = np.delete(X, list(notesDel), 2)
	Y = np.delete(Y, list(notesDel), 1)


	notesMap = set(range(129)).difference(notesDel)

	return X, Y, sorted(list(notesMap))


#This function creates samples out of each song
def createModelInputs(roll, seqLength=50, inc=1, padding=False):
	#roll is a list of numpy.array
	#split into arbitrary lenght sequences and extract next tone for a sequence (Y)
	#To do (idea): split into shorter melodies cutting any empty part that is long enough.
	X = []
	Y = []
	maxlength = max([len(s) for s in roll])-1
	minlength = min([len(s) for s in roll])-1
	for song in roll:
		#start (padding + seq)
		if padding == True:
			pos = 0
			empty = np.zeros((seqLength,128+1))
			while (pos < seqLength and pos < song.shape[0]):
				#zeros + part of seq
				sample = np.concatenate((empty[pos:],song[:pos]))
				X.append(sample)
				Y.append(song[pos])
				pos += inc

			#if seqLength is larger than song length
			if pos >= song.shape[0]:
				continue

		#mid
		pos = 0
		while pos+seqLength < song.shape[0]:
			sample = np.array(song[pos:pos+seqLength])
			X.append(sample)
			Y.append(song[pos+seqLength])
			pos += inc

		#don't implement end (seq + padding) because that could encourage stopping


	return np.array(X), np.array(Y)


def countDifferentTones(song):
	if len(song.shape) == 3 and song.shape[0] == 1:
		song = song[0]
	tones = 0
	for i in xrange(len(song)-1):
		if np.sum(song[i]-song[i+1]) > 0:
			tones += 1
	return tones

def expandInputs(songs, notesMap):
	songsList = []
	for i in xrange(len(songs)):
		song = np.zeros((songs[i].shape[0],128))
		for j in xrange(songs[i].shape[0]):
			for k in xrange(len(notesMap)):
				song[j][notesMap[k]] = songs[i][j][k]
		songsList.append(song)

def plotSong(song):
	plt.matshow(np.transpose(song))
	plt.colorbar()
	plt.show()

def plot2Songs(s1, s2):
	f, (ax1, ax2) = plt.subplots(2)
	m1 = ax1.matshow(np.transpose(s1))
	#ax1.set_title('Original song')
	ax2.matshow(np.transpose(s2))
	#ax2.set_title('Probabilities at middle layer')
	# Fine-tune figure; make subplots close to each other and hide x ticks for
	# all but bottom plot.
	f.subplots_adjust(right=0.8)
	cbar_ax = f.add_axes([0.85, 0.15, 0.05, 0.7])
	f.colorbar(m1,cax=cbar_ax)
	#plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)
	plt.show()



def plot3Songs(s1, s2, s3):
	f, (ax1, ax2, ax3) = plt.subplots(3)
	m1 = ax1.matshow(np.transpose(s1))
	#ax1.set_title('Original song')
	ax2.matshow(np.transpose(s2))
	#ax2.set_title('Probabilities at middle layer')
	ax3.matshow(np.transpose(s3))
	#ax3.set_title('Output song')
	# Fine-tune figure; make subplots close to each other and hide x ticks for
	# all but bottom plot.
	f.subplots_adjust(right=0.8)
	cbar_ax = f.add_axes([0.85, 0.15, 0.05, 0.7])
	f.colorbar(m1,cax=cbar_ax)
	#plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)
	plt.show()

def plotMK(mkmat):

	mat = np.zeros((mkmat.shape[1]*2, mkmat.shape[2]*2))
	mat[:mkmat.shape[1], :mkmat.shape[2]] = mkmat[0]
	mat[mkmat.shape[1]:, mkmat.shape[2]:] = mkmat[1]
	mat[:mkmat.shape[1], mkmat.shape[2]:] = mkmat[2]
	mat[mkmat.shape[1]:, :mkmat.shape[2]] = np.transpose(mkmat[2])

	plt.matshow(mat)
	plt.colorbar()
	plt.show()




In [198]:
roll = createRepresentation('C:/Users/Aaqui/Desktop/magenta/Database/Beethovan/', limitSongs=0, reductionRatio=128) #array of "piano roll" like representations

#Transform 
print("Creating output sequences...")
X, Y = createModelInputs(roll, padding=False, seqLength=50, inc=1)


Loading file 1: be_roman.mid
Loading file 2: beeth25.mid
Creating output sequences...


In [220]:
X_test = X[-200:]
X, Y, notesMap = compressInputs(X, Y)
input_dim = len(notesMap)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [225]:
model = Sequential()

model.add(Bidirectional(LSTM(10, return_sequences=True),input_shape=(5, 10)))
model.add(Dropout(0.6))
model.add(Bidirectional(LSTM(10)))
model.add(Dropout(0.6))
model.add(Dense(1))
model.add(Activation('softmax'))
model.compile(loss='mse', optimizer=optimizer)


NameError: name 'Bidirectional' is not defined

In [222]:
filepath="C:/Users/Aaqui/Untitled Folder/checkpoint_model_{epoch:02d}.hdf5"
model_save_callback = ModelCheckpoint(filepath, monitor='val_acc', 
                                      verbose=1, save_best_only=False, 
                                      mode='auto', period=5)

In [223]:
model.fit(np.array(X), np.array(y), 32, 20, verbose=1, callbacks=[model_save_callback])
model.save('C:/Users/Aaqui/Untitled Folder/Keras based/weights/weights1.h5')

ValueError: Error when checking input: expected lstm_7_input to have shape (30, 1) but got array with shape (50, 65)

In [128]:
from keras.models import load_model as ld
trained_model = ld('C:/Users/Aaqui/Untitled Folder/Keras based/weights/weights1.h5')

In [129]:
prediction = trained_model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]

In [130]:
mid = MidiFile()
track = MidiTrack()
t = 0
for note in prediction:
    # 147 means note_on
    # 67 is velocity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('LSTM_music1.mid')

In [32]:
#import _pickle as cPickle
#file = open('C:/Users/Aaqui/Untitled Folder/Keras based/JSB Chorales.pickle','rb')
#dataset = cPickle.load(file)


In [55]:
#len(dataset['test'])



77

In [60]:
#(dataset['test'][0])

[(72, 76, 79, 84),
 (72, 76, 79, 84),
 (71, 74, 79, 86),
 (73, 77, 81, 88),
 (75, 81, 83, 90),
 (75, 81, 83, 90),
 (),
 (71, 75, 83, 90),
 (76, 83, 91),
 (71, 79, 83, 86),
 (67, 79, 83, 86),
 (64, 79, 83, 89),
 (60, 79, 84, 88),
 (60, 79, 84, 88),
 (60, 79, 84, 88),
 (67, 79, 83, 86),
 (72, 79, 84, 88),
 (73, 82, 88, 91),
 (74, 81, 86, 91),
 (62, 74, 84, 90),
 (67, 74, 83, 91),
 (67, 74, 83, 91),
 (72, 76, 79, 84),
 (72, 76, 79, 84),
 (71, 74, 79, 86),
 (70, 72, 79, 88),
 (69, 72, 81, 90),
 (69, 72, 81, 90),
 (),
 (62, 74, 81, 90),
 (67, 74, 83, 91),
 (67, 79, 83, 86),
 (71, 79, 86),
 (69, 79, 83, 89),
 (72, 79, 84, 88),
 (72, 79, 84, 88),
 (),
 (72, 78, 81, 86),
 (70, 79, 82, 88),
 (67, 82, 86, 91),
 (74, 81, 86, 91),
 (62, 74, 84, 90),
 (67, 74, 83, 91),
 (67, 74, 83, 91),
 (),
 (67, 79, 83, 86),
 (71, 79, 86),
 (71, 79, 86),
 (72, 79, 84, 88),
 (72, 79, 86),
 (69, 81, 84, 89),
 (72, 79, 84, 88),
 (67, 79, 84, 88),
 (67, 78, 84, 88),
 (67, 79, 83, 86),
 (67, 79, 83, 86),
 (),
 (72, 7